# TD2 part 2: Named entity recognition

Dans ce TD, nous allons prendre un datasets où les noms de personnes sont taggés.<br>
Nous allons transformer ces données en tenseurs X, y et attention_mask.<br>
Nous allons créer un modèle RNN pour prédire si un mot est un nom de personne.<br>
Nous allons ensuite créer la loop avec l'optimizer pour apprendre le modèle.<br>
Du modèle appris (prédisant sur les tokens), nous allons postprocess les prédictions pour avoir les prédictions sur les noms.

Un fois que la loop est créée et que le modèle apprend, nous allons changer la structure du modèle:
- Changer learning rate. Comment se comporte le modèle
- Ajouter des couches denses, ReLU, dropout, normalization
- Changer le nombre de layers du RNN, LSTM.

Lorsqu'on a un bon modèle de prédiction pour les noms de personnes, nous allons l'appliquer à notre projet fil rouge.
Utilisez-le tel que. Quelle accuracy ?
Ré-entrainez la (les) dernière(s) couche(s) du modèle sur notre jeu de données. A-t-il gagné en accuracy ?

In [1]:
# Import
import matplotlib.pyplot as plt
import numpy as np
import torch
from tqdm.notebook import tqdm
import transformers

## Data

Télécharger le dataset MultiNERD FR [ici](https://github.com/Babelscape/multinerd)<br>
Mettez les données dans le dossier data/raw du projet.


In [2]:
def extract_multinerd_person_words(filename="../data/raw/train_fr.tsv"):
    with open(filename) as f:
        tagged_words = [line.strip().split("\t") for line in f]
        
        # Joining words until we meet a dot
        # Word's label is 1 if 'PER' is in its tag
        sentences = []
        sentence_labels = []
    
        this_word = []
        this_labels = []
        for tagged_word in tagged_words:
            if len(tagged_word) < 3:
                # not a tagged word
                continue
            word = tagged_word[1]
            tag = tagged_word[2]
        
            if word == '.':
                sentences.append(" ".join(this_word))
                sentence_labels.append(this_labels)
            
                this_word = []
                this_labels = []
            else:
                this_word.append(word)
                this_labels.append(1 * tag.endswith("PER"))

    return sentences, sentence_labels


In [3]:
sentences, labels = extract_multinerd_person_words("../src/data/raw/td2/train_fr.tsv")

## Tokenizer

En utilisant le tokenizer d'HuggingFace "camembert-base":
- Transformer les phrases en tokens
- Obtenez des vecteur y qui ont le même nombre d'entrées qu'il y a de tokens dans la phrase
- Ayez un tenseur "attention_mask" pour savoir sur quels tokens on cherche à predire le label
- Transformez les tokens en token_ids (avec le tokenizer)
Avec tout cela, vous pouvez former vos tenseurs X, Y et attention_mask

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("camembert-base")

In [5]:
def build_tokens_and_labels_and_attention_mask(tokenizer, sentence, labels):
    words = sentence.split()
    tokens = []
    tokens_label = []
    attention_mask = []
    
    for word, label in zip(words, labels):
        this_tokens = tokenizer.tokenize(word)
        tokens += this_tokens
        
        this_labels = [0] * len(this_tokens)
        this_labels[0] = label        
        tokens_label += this_labels
        
        this_attention_mask = [1] + [0] * (len(this_tokens) - 1)
        attention_mask += this_attention_mask
        
    return tokens, tokens_label, attention_mask

In [6]:
tokens, label, padding_masks = build_tokens_and_labels_and_attention_mask(tokenizer, sentences[0], labels[0])

In [7]:
def build_X_y_attention_mask(tokenizer, sentences, labels):
    tokens = []
    tokens_label = []
    attention_mask = []
    
    for sentence, sentence_labels in tqdm(zip(sentences, labels), total=len(sentences)):
        this_tokens, this_tokens_label, this_attention_mask = build_tokens_and_labels_and_attention_mask(tokenizer, sentence, sentence_labels)
        tokens += this_tokens
        tokens_label += this_tokens_label
        attention_mask += this_attention_mask
        
    X = torch.tensor(tokenizer.convert_tokens_to_ids(tokens))
    y = torch.tensor(tokens_label)
    attention_mask = torch.tensor(attention_mask)
    
    return X, y, attention_mask

In [8]:
X, y, attention_mask = build_X_y_attention_mask(tokenizer, sentences, labels)

  0%|          | 0/140436 [00:00<?, ?it/s]

In [9]:
y[:10]

tensor([0, 0, 0, 0, 1, 0, 1, 0, 0, 0])

## Model

Contruisez un modèle RNN comme dans la partie 1. Pour l'instant, il prendra comme arguments:
- Vocab size: le nombre de différents tokens du tokenizer (52 000 pour camembert-base)
- Embedding dim: la dimension de l'embedding des tokens (par défaut 50)
- hidden_dim: la dimension de l'état récurrent de votre RNN (par défaut 20)
- tagset_size: la nombre de classes possibles pour les prédictions (ici 2)

Dans le forward, votre modèle enchaînera les couches suivantes:
- Un embedding
- Un RNN
- Un ReLU
- Une couche linéaire
- Un softmax pour que la somme des prédictions pour une entrée soit égale à 1 (la prédiction pour un élément et sa probabilité d'être dans chaque classe)

In [10]:
class RNN(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim=128, hidden_dim=64, tagset_size=2, RNN=torch.nn.LSTM):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embedding_dim)
        self.rnn = RNN(embedding_dim, hidden_dim)
        self.relu = torch.nn.ReLU()
        self.linear = torch.nn.Linear(hidden_dim, tagset_size)
        self.softmax = torch.nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.rnn(x)
        x = self.relu(x)
        x = self.linear(x)
        x = self.softmax(x)
        return x

## Optimizer

Je fournis ici une fonction prenant un modèle, des tenseurs X, y, attention_mask.
Pour chaque batch:
- La loop utilise le modèle pour prédire sur x_batch
- Avec attention_mask, elle identifie sur quels tokens les prédictions compte
- Elle regarde la cross entropy entre y\[attention_ids\] et yhat\[attention_ids\]
- Elle output un dictionnaire avec le model et la loss au fur et à mesure des itérations

Entraînez le modèle avec vos données. <br>
Plottez la loss history.<br>
Itérez sur le modèle pour l'améliorer:
- Changer learning rate. Comment se comporte le modèle
- Ajouter des couches denses, ReLU, dropout, normalization
- Changer le nombre de layers du RNN, LSTM.



In [11]:
def train_model(model, X, y, attention_masks, n_epochs=100, lr=0.05, batch_size=128):
    loss_function = torch.nn.CrossEntropyLoss()
    loss_history = []

    dataset = torch.utils.data.TensorDataset(X, y, attention_masks)
    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(n_epochs):
        iterator = tqdm(enumerate(loader), total=len(loader))
        for i, (x_batch, y_batch, mask) in iterator:
            x_batch, y_batch, mask = x_batch.cuda(), y_batch.cuda(), mask.cuda()
            optimizer.zero_grad()
        
            ids = mask.reshape(-1)
            yhat = model(x_batch).reshape((-1, 2))[ids]
            this_y = y_batch.reshape(-1)[ids]
            
            loss = loss_function(yhat, this_y)
            loss.backward()
            
            loss_history.append(loss.cpu().clone().detach())
        
            optimizer.step()
            
            s = np.round((np.mean(loss_history[-10:])), 2)
            iterator.set_description(f"Got loss at {epoch} {s}      ")
    return {"model": model, "loss_history": loss_history}

In [12]:
model = RNN(len(tokenizer))
model = model.cuda()

In [ ]:
result = train_model(model, X, y, attention_mask.cuda(), n_epochs=100, lr=0.001, batch_size=256)

  0%|          | 0/16227 [00:00<?, ?it/s]

  0%|          | 0/16227 [00:00<?, ?it/s]

  0%|          | 0/16227 [00:00<?, ?it/s]

## Postprocessing

Créer une fonction prenant les prédictions du modèle (au niveau token) et sort les prédictions au niveau mot.<br>
Par exemple, admettons que, pour un mot, la prédiction du 1er token est la seule qu'on considère.<br>
si la phrase est "Bonjour John", avec les tokens \["bon", "jour", "Jo", "hn"\] avec les predictions \[0.12, 0.65, 0.88, 0.45\]<br>
Je veux récupérer les prédictions "bonjour": 0.12, "John": 0.88

In [ ]:
def postprocess_predictions(predictions, attention_mask):
    predictions = predictions.cpu().detach().numpy()
    attention_mask = attention_mask.cpu().detach().numpy()
    
    predictions = predictions[attention_mask == 1]
    predictions = predictions[:, 1]
    
    return predictions

In [ ]:
predictions = postprocess_predictions(result["model"](X.cuda()), attention_mask.cuda())